In [3]:
import sys
import os

sys.path.append(os.path.abspath("../.."))

from tiny_graph.graph.executable import Graph
from tiny_graph.models.base import GraphState
from tiny_graph.buffer.factory import History, LastValue
from tiny_graph.checkpoint.storage.local_storage import LocalStorage
from tiny_graph.constants import START, END
import time
from typing import Dict
from rich import print as rprint


# Define our state model
class ProcessState(GraphState):
    status: LastValue[str]
    results: History[Dict[str, float]]


# Initialize state and graph with local storage and chain_id
chain_id = "process_workflow_v1"
state = ProcessState(status="", results={})
graph = Graph(state=state, chain_id=chain_id, checkpoint_storage=LocalStorage)


# Define processing nodes
@graph.node()
def initialize_process(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "initializing"}


@graph.node()
def process_data_1(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_1", "results": {"accuracy": 0.85, "step": 1.0}}


@graph.node()
def process_data_2(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_2", "results": {"accuracy": 0.92, "step": 2.0}}


@graph.node()
def finalize(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "completed"}


# Create the workflow
graph.add_edge(START, "initialize_process")
graph.add_edge("initialize_process", "process_data_1")
graph.add_edge("process_data_1", "process_data_2")
graph.add_edge("process_data_2", "finalize")
graph.add_edge("finalize", END)

# Compile and execute
graph.compile()


def run_workflow():
    print("Starting execution...")
    print(f"Using chain_id: {graph.chain_id}")

    # Execute the graph (checkpoints will be automatically saved)
    graph.execute()

    print("\nExecution completed!")
    print(f"Final status: {graph.state.status}")
    print(f"Results history: {graph.state.results}")

    print("\nVerifying final state from checkpoint:")
    rprint(f"Checkpoints: {graph.checkpoint_storage._storage}")


# Run the example
if __name__ == "__main__":
    run_workflow()

DEBUG:tiny_graph.graph.executable:Executing node: initialize_process
DEBUG:tiny_graph.graph.executable:Executing task in node: initialize_process


Starting execution...
Using chain_id: process_workflow_v1


INFO:tiny_graph.checkpoint.storage.local_storage:Checkpoint 'process_workflow_v1_f544fe10-404b-41c8-8e88-61f3aa87beff' saved in memory.
DEBUG:tiny_graph.graph.executable:State updated after node: initialize_process
DEBUG:tiny_graph.graph.executable:Executing node: process_data_1
DEBUG:tiny_graph.graph.executable:Executing task in node: process_data_1
INFO:tiny_graph.checkpoint.storage.local_storage:Checkpoint 'process_workflow_v1_406286ae-b170-4a7b-a329-4819473b86c1' saved in memory.
DEBUG:tiny_graph.graph.executable:State updated after node: process_data_1
DEBUG:tiny_graph.graph.executable:Executing node: process_data_2
DEBUG:tiny_graph.graph.executable:Executing task in node: process_data_2
INFO:tiny_graph.checkpoint.storage.local_storage:Checkpoint 'process_workflow_v1_862203f3-082f-467a-b2d9-e96aa3c6cb87' saved in memory.
DEBUG:tiny_graph.graph.executable:State updated after node: process_data_2
DEBUG:tiny_graph.graph.executable:Executing node: finalize
DEBUG:tiny_graph.graph.execu


Execution completed!
Final status: completed
Results history: [{'accuracy': 0.85, 'step': 1.0}, {'accuracy': 0.92, 'step': 2.0}]

Verifying final state from checkpoint:


Checkpoints: defaultdict(<class 'dict'>, {'process_workflow_v1': 
{'process_workflow_v1_f544fe10-404b-41c8-8e88-61f3aa87beff': {'chain_id': 'process_workflow_v1', 'model_class': 
<class '__main__.ProcessState'>, 'model_version': '2c56039250db38e7d4c79d6f205816f8', 'data': 
'{"version":"2c56039250db38e7d4c79d6f205816f8","status":"initializing","results":{}}', 'timestamp': 
datetime.datetime(2024, 11, 17, 12, 11, 53, 726645)}, 'process_workflow_v1_406286ae-b170-4a7b-a329-4819473b86c1': 
{'chain_id': 'process_workflow_v1', 'model_class': <class '__main__.ProcessState'>, 'model_version': 
'2c56039250db38e7d4c79d6f205816f8', 'data': 
'{"version":"2c56039250db38e7d4c79d6f205816f8","status":"processing_1","results":[{"accuracy":0.85,"step":1.0}]}', 
'timestamp': datetime.datetime(2024, 11, 17, 12, 11, 54, 234150)}, 
'process_workflow_v1_862203f3-082f-467a-b2d9-e96aa3c6cb87': {'chain_id': 'process_workflow_v1', 'model_class': 
<class '__main__.ProcessState'>, 'model_version': '2c56039250db38e7d4c79d6f205816f8', 'data': 
'{"version":"2c56039250db38e7d4c79d6f205816f8","status":"processing_2","results":[{"accuracy":0.85,"step":1.0},{"ac
curacy":0.92,"step":2.0}]}', 'timestamp': datetime.datetime(2024, 11, 17, 12, 11, 54, 742470)}, 
'process_workflow_v1_3cb99c3b-933b-4f8c-9243-d90ded2c6bea': {'chain_id': 'process_workflow_v1', 'model_class': 
<class '__main__.ProcessState'>, 'model_version': '2c56039250db38e7d4c79d6f205816f8', 'data': 
'{"version":"2c56039250db38e7d4c79d6f205816f8","status":"completed","results":[{"accuracy":0.85,"step":1.0},{"accur
acy":0.92,"step":2.0}]}', 'timestamp': datetime.datetime(2024, 11, 17, 12, 11, 55, 255218)}}})

In [ ]:
graph.checkpoint_storage._storage

In [ ]:
state.model_fields.keys()